In [20]:
# Install required libraries (run in your environment if needed)
# !pip install faker reportlab

from faker import Faker
from reportlab.lib.pagesizes import letter
from reportlab.lib import colors
from reportlab.lib.units import inch
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph, Spacer, Image, KeepTogether
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from datetime import datetime, timedelta
import random
import os

# Ensure output directory exists
output_dir = "out"
os.makedirs(output_dir, exist_ok=True)

# Initialize Faker
fake = Faker()


In [21]:

# Unified synthetic data generation for Citibank
def generate_statement_data():
    """Generate synthetic data compatible with Citibank Classic Style"""
    start = datetime(2025, 6, 1)
    end = datetime(2025, 6, 30)
    statement_period = f"{start.strftime('%B %d, %Y')} - {end.strftime('%B %d, %Y')}"
    account_type = random.choice(["Citi Access Checking", "CitiBusiness Checking"])
    account_holder = fake.name()
    account_holder_address_lines = [fake.street_address(), f"{fake.city()}, {fake.state_abbr()} {fake.zipcode()}"]
    customer_iban = fake.iban()
    customer_account_number = fake.bban()
    client_number = fake.ssn()
    date_of_birth = fake.date_of_birth(minimum_age=18, maximum_age=80).strftime("%m/%d/%Y")
    
    # Generate transactions (deposits and withdrawals)
    deposit_descriptions = ["Direct Deposit", "Salary Deposit", "Online Transfer", "Cash Deposit"]
    withdrawal_descriptions = ["Payment", "Grocery Shopping", "Utility Payment", "Online Purchase"]
    transactions = []
    beginning_balance = round(random.uniform(2000, 10000), 2)
    current_balance = beginning_balance
    
    # Generate deposits
    deposits = [
        {
            "date": (start + timedelta(days=random.randint(0, 29))).strftime("%m/%d/%Y"),
            "description": f"{fake.company()} {random.choice(deposit_descriptions)}",
            "amount": round(random.uniform(300, 5000), 2)
        } for _ in range(random.randint(2, 5))
    ]
    
    # Generate withdrawals
    withdrawals = [
        {
            "date": (start + timedelta(days=random.randint(0, 29))).strftime("%m/%d/%Y"),
            "description": f"{fake.company()} {random.choice(withdrawal_descriptions)}",
            "amount": -round(random.uniform(40, 1000), 2)
        } for _ in range(random.randint(3, 7))
    ]
    
    # Combine and sort transactions by date
    for item in deposits + withdrawals:
        transactions.append({
            "date": item["date"],
            "description": item["description"],
            "debit": f"£{abs(item['amount']):,.2f}" if item["amount"] < 0 else "",
            "credit": f"£{item['amount']:,.2f}" if item["amount"] >= 0 else "",
            "balance": current_balance + item["amount"]
        })
        current_balance += item["amount"]
        transactions[-1]["balance"] = f"£{current_balance:,.2f}"
    
    transactions.sort(key=lambda x: datetime.strptime(x["date"], "%m/%d/%Y"))
    
    # Calculate totals
    total_debit = sum(abs(item["amount"]) for item in withdrawals)
    total_credit = sum(item["amount"] for item in deposits)
    total_balance = current_balance
    
    return {
        "logo": "sample_logos/citibank_logo.png",
        "account_type": account_type,
        "statement_period": statement_period,
        "statement_date": end.strftime("%B %d, %Y"),
        "account_holder": account_holder,
        "account_holder_address_lines": account_holder_address_lines,
        "account_holder_address": ", ".join(account_holder_address_lines),
        "customer_iban": customer_iban,
        "customer_account_number": customer_account_number,
        "client_number": client_number,
        "date_of_birth": date_of_birth,
        "customer_bank_name": "Citibank UK Limited",
        "transactions": transactions,
        "show_fee_waiver": random.choice([True, False]),
        "summary": {
            "opening_balance": f"£{beginning_balance:,.2f}",
            "total_debit": f"£{total_debit:,.2f}",
            "total_credit": f"£{total_credit:,.2f}",
            "total": f"£{total_balance:,.2f}",
            "transactions_count": len(transactions)
        }
    }

# Generate shared data
ctx = generate_statement_data()


In [ ]:
# ---------------------------------------------------------------------------------
# Citibank Classic Style
# ---------------------------------------------------------------------------------

def create_classic_statement(ctx):
    output_file = os.path.join(output_dir, "citibank_statement_classic.pdf")
    doc = SimpleDocTemplate(output_file, pagesize=letter, leftMargin=0.5*inch, rightMargin=0.5*inch, topMargin=0.5*inch, bottomMargin=0.5*inch)
    elements = []

    # Styles based on provided CSS, using Helvetica
    styles = getSampleStyleSheet()
    normal_style = ParagraphStyle(
        name='Normal', fontName='Helvetica', fontSize=9, leading=12.15, spaceAfter=4, textColor=colors.HexColor("#004685")
    )
    bold_style = ParagraphStyle(
        name='Bold', fontName='Helvetica-Bold', fontSize=9, leading=12.15, spaceAfter=4
    )
    header_style = ParagraphStyle(
        name='Header', fontName='Helvetica', fontSize=12, leading=16.2, spaceAfter=18, textTransform='uppercase', alignment=1
    )
    footer_style = ParagraphStyle(
        name='Footer', fontName='Helvetica', fontSize=7, leading=9.45, spaceAfter=6, alignment=4
    )
    centered_style = ParagraphStyle(
        name='Centered', fontName='Helvetica', fontSize=9, leading=12.15, spaceAfter=14, alignment=1
    )

    # Header
    PAGE_WIDTH, _ = letter
    header_content = []
    left_block = [
        Paragraph("Citigroup Centre", normal_style),
        Paragraph("33 Canada Square", normal_style),
        Paragraph("Canary Wharf", normal_style),
        Paragraph("London E14 5LB", normal_style),
        Paragraph("United Kingdom", normal_style),
        Paragraph("Contact us: citigroup.com/citi/contact.html", normal_style)
    ]
    right_block = []
    if os.path.exists(ctx['logo']):
        logo = Image(ctx['logo'], width=120, height=40)
        right_block.append(logo)
    
    header_table = Table(
        [[left_block, right_block]],
        colWidths=[PAGE_WIDTH * 0.45, PAGE_WIDTH * 0.45],
        rowHeights=[120]
    )
    header_table.setStyle(TableStyle([
        ('VALIGN', (0,0), (-1,-1), 'TOP'),
        ('ALIGN', (1,0), (1,0), 'RIGHT'),
        ('LEFTPADDING', (0,0), (-1,-1), 0),
        ('RIGHTPADDING', (0,0), (-1,-1), 0),
        ('TOPPADDING', (0,0), (-1,-1), 0),
        ('BOTTOMPADDING', (0,0), (-1,-1), 0),
    ]))
    elements.append(header_table)
    elements.append(Spacer(1, 12))
    elements.append(Table([[None]], colWidths=[PAGE_WIDTH * 0.9], rowHeights=[4], style=[
        ('BACKGROUND', (0,0), (-1,-1), colors.HexColor("#003e7e")),
        ('LEFTPADDING', (0,0), (-1,-1), 0),
        ('RIGHTPADDING', (0,0), (-1,-1), 0),
    ]))
    elements.append(Spacer(1, 12))

    # Bank and Customer Information
    info_data = [
        [
            [
                Paragraph("Bank information", bold_style),
                Paragraph(f"<b>Account Provider Name:</b> {ctx['customer_bank_name']}", normal_style),
                Paragraph(f"<b>Account Name:</b> {ctx['account_type']}", normal_style),
                Paragraph(f"<b>IBAN:</b> {ctx['customer_iban']}", normal_style),
                Paragraph(f"<b>Country code:</b> GB", normal_style),
                Paragraph(f"<b>Check Digits:</b> {ctx['customer_iban'][2:4]}", normal_style),
                Paragraph(f"<b>Bank code:</b> CITI", normal_style),
                Paragraph(f"<b>British bank code (sort code):</b> {ctx['customer_iban'][8:14]}", normal_style),
                Paragraph(f"<b>Bank account number:</b> {ctx['customer_account_number']}", normal_style),
            ],
            [
                Paragraph("Customer information", bold_style),
                Paragraph(f"<b>Client Name:</b> {ctx['account_holder']}", normal_style),
                Paragraph(f"<b>Client number ID:</b> {ctx['client_number']}", normal_style),
                Paragraph(f"<b>Date of birth:</b> {ctx['date_of_birth']}", normal_style),
                Paragraph(f"<b>Account number:</b> {ctx['customer_account_number']}", normal_style),
                Paragraph(f"<b>IBAN Bank:</b> {ctx['customer_iban']}", normal_style),
                Paragraph(f"<b>Bank name:</b> {ctx['customer_bank_name']}", normal_style),
            ]
        ]
    ]
    info_table = Table(info_data, colWidths=[PAGE_WIDTH * 0.432, PAGE_WIDTH * 0.432])
    info_table.setStyle(TableStyle([
        ('VALIGN', (0,0), (-1,-1), 'TOP'),
        ('LEFTPADDING', (0,0), (0,0), 0),
        ('RIGHTPADDING', (1,0), (1,0), 0),
        ('TOPPADDING', (0,0), (-1,-1), 0),
        ('BOTTOMPADDING', (0,0), (-1,-1), 0),
    ]))
    elements.append(info_table)
    elements.append(Spacer(1, 0.4*inch))

    # Important Account Information
    elements.append(Paragraph("Important Account Information", header_style))
    if ctx['account_type'] == "Citi Access Checking":
        info_text = (
            "Effective July 1, 2025, the monthly account fee for Citi Access Checking accounts will increase to £10 unless you maintain a minimum daily balance of £1,500 or have qualifying direct deposits of £500 or more per month.\n"
            "Starting June 30, 2025, Citibank will introduce real-time transaction alerts for Citi Access Checking accounts via the Citi Mobile UK app. Enable alerts at citibank.co.uk/alerts.\n"
            "Effective July 15, 2025, Citibank will waive overdraft fees for transactions of £5 or less and cap daily overdraft fees at two per day for Citi Access Checking accounts.\n"
            "For questions, visit citibank.co.uk or contact our Client Contact Centre at 0800 005 555 (or +44 20 7500 5500 from abroad), available 24/7."
        )
    else:
        info_text = (
            "Effective July 1, 2025, the monthly account fee for CitiBusiness Checking accounts will increase to £15 unless you maintain a minimum daily balance of £5,000 or have £2,000 in net purchases on a Citi Business Debit Card per month.\n"
            "Starting June 30, 2025, Citibank will offer enhanced cash flow tools for CitiBusiness Checking accounts via Citi Online Banking, including automated invoice tracking and payment scheduling.\n"
            "Effective July 15, 2025, Citibank will reduce domestic BACS transfer fees to £20 for CitiBusiness Checking accounts, down from £25.\n"
            "For questions, visit citibank.co.uk or contact our Client Contact Centre at 0800 005 555 (or +44 20 7500 5500 from abroad), available 24/7."
        )
    elements.append(Paragraph(info_text, normal_style))
    elements.append(Spacer(1, 0.4*inch))

    # Account Transactions
    elements.append(Paragraph("Account Transactions", header_style))
    right_info = [
        Paragraph(ctx['statement_period'], bold_style),
        Paragraph(f"Created on {ctx['statement_date']}", bold_style)
    ]
    right_table = Table([[right_info]], colWidths=[PAGE_WIDTH * 0.9])
    right_table.setStyle(TableStyle([
        ('ALIGN', (0,0), (-1,-1), 'RIGHT'),
        ('LEFTPADDING', (0,0), (-1,-1), 0),
        ('RIGHTPADDING', (0,0), (-1,-1), 0),
        ('TOPPADDING', (0,0), (-1,-1), 4),
        ('BOTTOMPADDING', (0,0), (-1,-1), 0),
    ]))
    elements.append(right_table)
    
    # Transactions Table
    transactions_data = [
        ["Transaction date", "Information", "Debit", "Credit", "", "Balance"]
    ]
    transactions_data.append([
        "", "Opening balance", "", "", "", ctx['summary']['opening_balance']
    ])
    for transaction in ctx['transactions']:
        transactions_data.append([
            transaction["date"],
            transaction["description"],
            transaction["debit"],
            transaction["credit"],
            "",
            transaction["balance"]
        ])
    transactions_data.append([
        "", "Total", ctx['summary']['total_debit'], ctx['summary']['total_credit'], "", ctx['summary']['total']
    ])
    transactions_table = Table(transactions_data, colWidths=[0.15*PAGE_WIDTH*0.9, 0.36*PAGE_WIDTH*0.9, 0.12*PAGE_WIDTH*0.9, 0.12*PAGE_WIDTH*0.9, 0.13*PAGE_WIDTH*0.9, 0.12*PAGE_WIDTH*0.9])
    transactions_table.setStyle(TableStyle([
        ('FONT', (0,0), (-1,0), 'Helvetica-Bold', 9),
        ('FONT', (0,1), (-1,-1), 'Helvetica', 9),
        ('BACKGROUND', (0,0), (-1,0), colors.HexColor("#d9d9d9")),
        ('ALIGN', (2,0), (3,-1), 'RIGHT'),
        ('ALIGN', (5,0), (5,-1), 'RIGHT'),
        ('LINEBELOW', (0,0), (-1,0), 1, colors.HexColor("#888888")),
        ('LINEBELOW', (0,1), (-1,-2), 1, colors.HexColor("#888888")),
        ('FONT', (0,1), (-1,1), 'Helvetica-Bold', 9),
        ('FONT', (0,-1), (-1,-1), 'Helvetica-Bold', 9),
        ('LEFTPADDING', (0,0), (-1,-1), 6),
        ('RIGHTPADDING', (0,0), (-1,-1), 6),
        ('TOPPADDING', (0,0), (-1,-1), 4),
        ('BOTTOMPADDING', (0,0), (-1,-1), 4),
    ]))
    elements.append(transactions_table)
    
    # Centered Notice
    elements.append(Paragraph(
        "This printout is for information purposes only. Your regular account statement of assets takes precedence.",
        centered_style
    ))
    
    # Footer
    elements.append(Spacer(1, 12))
    elements.append(Table([[None]], colWidths=[PAGE_WIDTH * 0.9], rowHeights=[4], style=[
        ('BACKGROUND', (0,0), (-1,-1), colors.HexColor("#003e7e")),
        ('LEFTPADDING', (0,0), (-1,-1), 0),
        ('RIGHTPADDING', (0,0), (-1,-1), 0),
    ]))
    footer_content = [
        Paragraph(
            "Citigroup UK Limited is authorised by the Prudential Regulation Authority and regulated by the Financial Conduct Authority and the Prudential Regulation Authority.",
            footer_style
        ),
        Paragraph(
            "Our firm’s Financial Services Register number is 805574. Citibank UK Limited is a company limited by shares registered in England and Wales with registered address at Citigroup Centre, Canada Square, Canary Wharf, London E14 5LB.",
            footer_style
        ),
        Paragraph(
            "© All rights reserved Citibank UK Limited 2021. CITI, the Arc Design & Citibank are registered service marks of Citigroup Inc. Calls may be monitored or recorded for training and service quality purposes. PNB FBD 132019.",
            footer_style
        )
    ]
    for para in footer_content:
        elements.append(para)
    elements.append(Paragraph("Citibank", ParagraphStyle('FooterRight', parent=footer_style, alignment=2)))

    # Build PDF
    doc.build(elements)
    print(f"PDF generated: {output_file}")

In [38]:
# ---------------------------------------------------------------------------------
# Citibank App Style
# ---------------------------------------------------------------------------------

def create_template1_statement(ctx):
    def create_card_content(content, width):
        # Center the content by adding equal padding on both sides
        total_padding = 40  # 20px left + 20px right from original CSS
        content_width = width - total_padding
        card_table = Table([[content]], colWidths=[content_width])
        card_table.setStyle(TableStyle([
            ('BACKGROUND', (0, 0), (-1, -1), colors.HexColor("#f2f6fb")),
            ('BOX', (0, 0), (-1, -1), 1, colors.HexColor("#e1e6ef")),
            ('ROUND', (0, 0), (-1, -1), 12),
            ('VALIGN', (0, 0), (-1, -1), 'TOP'),
            ('LEFTPADDING', (0, 0), (-1, -1), 20),  # Consistent 20pt padding on left
            ('RIGHTPADDING', (0, 0), (-1, -1), 20),  # Consistent 20pt padding on right
            ('TOPPADDING', (0, 0), (-1, -1), 20),
            ('BOTTOMPADDING', (0, 0), (-1, -1), 20),
            ('ALIGN', (0, 0), (-1, -1), 'CENTER'),  # Center the content horizontally
        ]))
        return KeepTogether(card_table)

    def get_styles():
        styles = getSampleStyleSheet()
        return {
            "appbar": ParagraphStyle(
                name='Appbar', fontName='Helvetica-Bold', fontSize=15, textColor=colors.white, leading=22.5
            ),
            "title": ParagraphStyle(
                name='Title', fontName='Helvetica', fontSize=15, textColor=colors.HexColor("#0066c1"), alignment=1, spaceAfter=12, leading=22.5
            ),
            "normal": ParagraphStyle(
                name='Normal', fontName='Helvetica', fontSize=15, textColor=colors.black, spaceAfter=4, leading=22.5
            ),
            "label": ParagraphStyle(
                name='Label', fontName='Helvetica-Bold', fontSize=15, textColor=colors.HexColor("#757575"), spaceAfter=4, leading=22.5
            ),
            "txn_head": ParagraphStyle(
                name='TxnHead', fontName='Helvetica-Bold', fontSize=15, textColor=colors.HexColor("#0066c1"), spaceAfter=6, leading=22.5
            ),
            "txn_row": ParagraphStyle(
                name='TxnRow', fontName='Helvetica', fontSize=10, textColor=colors.black, leading=15
            ),
            "highlight": ParagraphStyle(
                name='Highlight', fontName='Helvetica-Bold', fontSize=10, textColor=colors.black, leading=15
            ),
            "info": ParagraphStyle(
                name='Info', fontName='Helvetica', fontSize=12, textColor=colors.black, leading=18
            ),
            "notice": ParagraphStyle(
                name='Notice', fontName='Helvetica', fontSize=12, textColor=colors.HexColor("#757575"), alignment=1, spaceAfter=20, leading=18
            ),
            "footer": ParagraphStyle(
                name='Footer', fontName='Helvetica', fontSize=11, textColor=colors.HexColor("#757575"), alignment=1, spaceAfter=5, leading=16.5
            )
        }

    output_file = os.path.join(output_dir, f"citibank_statement_app_{ctx['customer_account_number'][-4:]}.pdf")
    doc = SimpleDocTemplate(output_file, pagesize=letter, leftMargin=0.5*inch, rightMargin=0.5*inch, topMargin=0.5*inch, bottomMargin=0.5*inch)
    elements = []
    styles = get_styles()
    PAGE_WIDTH, _ = letter
    WRAPPER_WIDTH = 480  # Max-width from CSS

    # App Bar (Header with light blue background and centered logo)
    appbar_content = []
    if os.path.exists(ctx['logo']):
        logo = Image(ctx['logo'], width=120, height=40)  # Increased size for better visibility
        appbar_content.append(logo)
    appbar_table = Table([appbar_content], colWidths=[WRAPPER_WIDTH])
    appbar_table.setStyle(TableStyle([
        ('BACKGROUND', (0,0), (-1,-1), colors.HexColor("#ADD8E6")),  # Light blue background
        ('VALIGN', (0,0), (-1,-1), 'MIDDLE'),
        ('ALIGN', (0,0), (-1,-1), 'CENTER'),  # Center the logo
        ('LEFTPADDING', (0,0), (-1,-1), 20),
        ('RIGHTPADDING', (0,0), (-1,-1), 20),
        ('TOPPADDING', (0,0), (-1,-1), 14),
        ('BOTTOMPADDING', (0,0), (-1,-1), 14),
    ]))
    elements.append(appbar_table)
    elements.append(Spacer(1, 16))

    # Account Card
    account_content = [
        Paragraph("Account", styles["title"]),
        Paragraph(f"<b>Holder:</b> {ctx['account_holder']}", styles["normal"]),
        Paragraph(f"<b>Account #:</b> {ctx['customer_account_number']}", styles["normal"]),
        Paragraph(f"<b>IBAN:</b> {ctx['customer_iban']}", styles["normal"]),
        Paragraph(f"<b>Bank:</b> {ctx['customer_bank_name']}", styles["normal"]),
        Paragraph(f"<b>Type:</b> {ctx['account_type']}", styles["normal"])
    ]
    elements.append(create_card_content(account_content, WRAPPER_WIDTH))
    elements.append(Spacer(1, 20))

    # Period Card
    period_content = [
        Paragraph("Period", styles["title"]),
        Paragraph(ctx['statement_period'], styles["normal"]),
        Paragraph(f"Generated: {ctx['statement_date']}", styles["normal"])
    ]
    elements.append(create_card_content(period_content, WRAPPER_WIDTH))
    elements.append(Spacer(1, 20))

    # Important Account Information Card
    info_content = [Paragraph("Important Account Information", styles["title"])]
    if ctx['account_type'] == "Citi Access Checking":
        info_text = (
            "Effective July 1, 2025, the monthly account fee for Citi Access Checking accounts will increase to £10 unless you maintain a minimum daily balance of £1,500 or have qualifying direct deposits of £500 or more per month.\n"
            "Starting June 30, 2025, Citibank will introduce real-time transaction alerts for Citi Access Checking accounts via the Citi Mobile UK app. Enable alerts at citibank.co.uk/alerts.\n"
            "Effective July 15, 2025, Citibank will waive overdraft fees for transactions of £5 or less and cap daily overdraft fees at two per day for Citi Access Checking accounts.\n"
            "For questions, visit citibank.co.uk or contact our Client Contact Centre at 0800 005 555 (or +44 20 7500 5500 from abroad), available 24/7."
        )
    else:
        info_text = (
            "Effective July 1, 2025, the monthly account fee for CitiBusiness Checking accounts will increase to £15 unless you maintain a minimum daily balance of £5,000 or have £2,000 in net purchases on a Citi Business Debit Card per month.\n"
            "Starting June 30, 2025, Citibank will offer enhanced cash flow tools for CitiBusiness Checking accounts via Citi Online Banking, including automated invoice tracking and payment scheduling.\n"
            "Effective July 15, 2025, Citibank will reduce domestic BACS transfer fees to £20 for CitiBusiness Checking accounts, down from £25.\n"
            "For questions, visit citibank.co.uk or contact our Client Contact Centre at 0800 005 555 (or +44 20 7500 5500 from abroad), available 24/7."
        )
    info_content.extend([
        Table([[None]], colWidths=[WRAPPER_WIDTH - 40], rowHeights=[1], style=[
            ('BACKGROUND', (0,0), (-1,-1), colors.HexColor("#0066c1")),
        ]),
        Paragraph(info_text, styles["info"]),
        Table([[None]], colWidths=[WRAPPER_WIDTH - 40], rowHeights=[1], style=[
            ('BACKGROUND', (0,0), (-1,-1), colors.HexColor("#0066c1")),
        ])
    ])
    elements.append(create_card_content(info_content, WRAPPER_WIDTH))
    elements.append(Spacer(1, 20))

    # Transactions Card
    transactions_content = [Paragraph("Transactions", styles["title"])]
    txn_head_data = [
        [Paragraph("Date", styles["txn_head"]), Paragraph("Description", styles["txn_head"]), Paragraph("Amount", styles["txn_head"])]
    ]
    txn_head_table = Table(txn_head_data, colWidths=[90, 200, 70])  # Adjusted middle column to 200 points
    txn_head_table.setStyle(TableStyle([
        ('LEFTPADDING', (0,0), (-1,-1), 6),
        ('RIGHTPADDING', (0,0), (-1,-1), 6),
        ('TOPPADDING', (0,0), (-1,-1), 0),
        ('BOTTOMPADDING', (0,0), (-1,-1), 6),
        ('ALIGN', (2,0), (2,0), 'RIGHT'),
    ]))
    transactions_content.append(txn_head_table)
    
    # Opening Balance
    opening_row = [
        Paragraph("", styles["highlight"]),
        Paragraph("Opening balance", styles["highlight"]),
        Paragraph(ctx['summary']['opening_balance'], styles["highlight"])
    ]
    opening_table = Table([opening_row], colWidths=[90, 200, 70])  # Adjusted middle column to 200 points
    opening_table.setStyle(TableStyle([
        ('BACKGROUND', (0,0), (-1,-1), colors.HexColor("#0066c1")),
        ('BOX', (0,0), (-1,-1), 1, colors.HexColor("#0066c1")),
        ('ROUND', (0,0), (-1,-1), 8),
        ('LEFTPADDING', (0,0), (1,0), 10),  # Shift middle column left by 10 points
        ('RIGHTPADDING', (0,0), (-1,-1), 12),
        ('TOPPADDING', (0,0), (-1,-1), 10),
        ('BOTTOMPADDING', (0,0), (-1,-1), 10),
        ('ALIGN', (2,0), (2,0), 'RIGHT'),
    ]))
    transactions_content.append(opening_table)
    
    # Transactions
    for transaction in ctx['transactions']:
        # Truncate description to 25 characters with ellipsis if needed
        desc = transaction["description"]
        if len(desc) > 25:
            desc = desc[:25] + "..."
        amount = f"-{transaction['debit']}" if transaction['debit'] else f"+{transaction['credit']}"
        txn_row = [
            Paragraph(transaction["date"], styles["txn_row"]),
            Paragraph(desc, styles["txn_row"]),
            Paragraph(amount, styles["txn_row"])
        ]
        txn_table = Table([txn_row], colWidths=[90, 200, 70])  # Adjusted middle column to 200 points
        txn_table.setStyle(TableStyle([
            ('LINEBELOW', (0,0), (-1,-1), 1, colors.HexColor("#e1e6ef")),
            ('LEFTPADDING', (0,0), (-1,-1), 6),
            ('RIGHTPADDING', (0,0), (-1,-1), 6),
            ('TOPPADDING', (0,0), (-1,-1), 8),
            ('BOTTOMPADDING', (0,0), (-1,-1), 8),
            ('ALIGN', (2,0), (2,0), 'RIGHT'),
        ]))
        transactions_content.append(txn_table)
    
    # Total
    total_row = [
        Paragraph("", styles["highlight"]),
        Paragraph("Total", styles["highlight"]),
        Paragraph(ctx['summary']['total'], styles["highlight"])
    ]
    total_table = Table([total_row], colWidths=[90, 200, 70])  # Adjusted middle column to 200 points
    total_table.setStyle(TableStyle([
        ('BACKGROUND', (0,0), (-1,-1), colors.HexColor("#0066c1")),
        ('BOX', (0,0), (-1,-1), 1, colors.HexColor("#0066c1")),
        ('ROUND', (0,0), (-1,-1), 8),
        ('LEFTPADDING', (0,0), (1,0), 10),  # Shift middle column left by 10 points
        ('RIGHTPADDING', (0,0), (-1,-1), 12),
        ('TOPPADDING', (0,0), (-1,-1), 10),
        ('BOTTOMPADDING', (0,0), (-1,-1), 10),
        ('ALIGN', (2,0), (2,0), 'RIGHT'),
    ]))
    transactions_content.append(total_table)
    
    elements.append(create_card_content(transactions_content, WRAPPER_WIDTH))
    elements.append(Spacer(1, 20))

    # Notice
    elements.append(Paragraph(
        "This printout is for information purposes only. Your regular account statement of assets takes precedence.",
        styles["notice"]
    ))

    # Footer
    footer_content = [
        Paragraph("Disclosures", styles["footer"]),
        Paragraph(
            "Citibank UK Limited is authorised by the Prudential Regulation Authority and regulated by the Financial Conduct Authority and the Prudential Regulation Authority.",
            styles["footer"]
        ),
        Paragraph(
            "Our firm’s Financial Services Register number is 805574. Citibank UK Limited is a company limited by shares registered in England and Wales with registered address at Citigroup Centre, Canada Square, Canary Wharf, London E14 5LB.",
            styles["footer"]
        ),
        Paragraph(
            "© 2025 Citibank UK Limited – Calls may be monitored or recorded for training and service quality purposes. CITI, the Arc Design & Citibank are registered service marks of Citigroup Inc.",
            styles["footer"]
        ),
        Paragraph(
            "Deposits are protected by the Financial Services Compensation Scheme (FSCS) up to £85,000 per depositor, subject to eligibility. For details on account terms or overdraft policies, visit citibank.co.uk or call 0800 005 555 (UK) or +44 20 7500 5500 (international).",
            styles["footer"]
        )
    ]
    footer_table = Table([[footer_content]], colWidths=[WRAPPER_WIDTH])
    footer_table.setStyle(TableStyle([
        ('LEFTPADDING', (0,0), (-1,-1), 12),
        ('RIGHTPADDING', (0,0), (-1,-1), 12),
        ('TOPPADING', (0,0), (-1,-1), 24),
        ('BOTTOMPADDING', (0,0), (-1,-1), 32),
    ]))
    elements.append(footer_table)

    # Build PDF
    doc.build(elements)
    print(f"PDF generated: {output_file}")

In [62]:
# ---------------------------------------------------------------------------------
# Citibank Alternate Style
# ---------------------------------------------------------------------------------

def create_template2_statement(ctx):
    output_file = os.path.join(output_dir, f"citibank_statement_alternate_{ctx['customer_account_number'][-4:]}.pdf")
    doc = SimpleDocTemplate(output_file, pagesize=letter, leftMargin=0.5*inch, rightMargin=0.5*inch, topMargin=0.5*inch, bottomMargin=0.5*inch)
    elements = []
    
    # Styles based on provided CSS, using Helvetica
    styles = getSampleStyleSheet()
    header_style = ParagraphStyle(
        name='Header', fontName='Helvetica', fontSize=18, textColor=colors.white, fontWeight=600, leading=27
    )
    title_style = ParagraphStyle(
        name='Title', fontName='Helvetica', fontSize=12, textColor=colors.HexColor("#003087"), alignment=1, spaceAfter=10, leading=18
    )
    bold_style = ParagraphStyle(
        name='Bold', fontName='Helvetica-Bold', fontSize=12, textColor=colors.HexColor("#003087"), spaceAfter=5, leading=18
    )
    normal_style = ParagraphStyle(
        name='Normal', fontName='Helvetica', fontSize=9, textColor=colors.HexColor("#333"), spaceAfter=3, leading=13.5  # Reduced to 9pt
    )
    info_style = ParagraphStyle(
        name='Info', fontName='Helvetica', fontSize=10, textColor=colors.HexColor("#333"), spaceAfter=5, leading=15
    )
    meta_style = ParagraphStyle(
        name='Meta', fontName='Helvetica-Bold', fontSize=12, textColor=colors.HexColor("#333"), alignment=2, spaceAfter=10, leading=18
    )
    notice_style = ParagraphStyle(
        name='Notice', fontName='Helvetica', fontSize=11, textColor=colors.HexColor("#666"), alignment=1, spaceAfter=20, leading=16.5
    )
    footer_style = ParagraphStyle(
        name='Footer', fontName='Helvetica', fontSize=10, textColor=colors.HexColor("#666"), spaceAfter=5, leading=15
    )
    footer_right_style = ParagraphStyle(
        name='FooterRight', fontName='Helvetica', fontSize=10, textColor=colors.HexColor("#666"), alignment=2, spaceAfter=5, leading=15
    )
    
    PAGE_WIDTH, _ = letter
    
    # Header
    header_content = []
    if os.path.exists(ctx['logo']):
        logo = Image(ctx['logo'], width=120, height=40)
        header_content.append(logo)
    header_table = Table([header_content], colWidths=[PAGE_WIDTH - 60])
    header_table.setStyle(TableStyle([
        ('BACKGROUND', (0,0), (-1,-1), colors.HexColor("#D3D3D3")),
        ('VALIGN', (0,0), (-1,-1), 'CENTER'),
        ('ALIGN', (0,0), (-1,-1), 'RIGHT'),
        ('LEFTPADDING', (0,0), (-1,-1), 15),
        ('RIGHTPADDING', (0,0), (-1,-1), 15),
        ('TOPPADDING', (0,0), (-1,-1), 10),
        ('BOTTOMPADDING', (0,0), (-1,-1), 10),
    ]))
    elements.append(header_table)
    elements.append(Spacer(1, 20))
    
    # Content Container (white background with border)
    content_table_style = TableStyle([
        ('BACKGROUND', (0,0), (-1,-1), colors.white),
        ('BOX', (0,0), (-1,-1), 1, colors.HexColor("#ddd")),
        ('LEFTPADDING', (0,0), (-1,-1), 15),
        ('RIGHTPADDING', (0,0), (-1,-1), 15),
        ('TOPPADDING', (0,0), (-1,-1), 15),
        ('BOTTOMPADDING', (0,0), (-1,-1), 15),
    ])
    
    # Bank Information
    bank_info = [
        Paragraph("Bank Information", bold_style),
        Paragraph(f"Account Provider Name: {ctx['customer_bank_name']}", normal_style),
        Paragraph(f"Account Name: {ctx['account_type']}", normal_style),
        Paragraph(f"IBAN: {ctx['customer_iban']}", normal_style),
        Paragraph(f"Country code: GB", normal_style),
        Paragraph(f"Check Digits: {ctx['customer_iban'][2:4]}", normal_style),
        Paragraph(f"Bank code: CITI", normal_style),
        Paragraph(f"Sort code: {ctx['customer_iban'][8:14]}", normal_style),
        Paragraph(f"Account number: {ctx['customer_account_number']}", normal_style)
    ]
    bank_table = Table([[bank_info]], colWidths=[PAGE_WIDTH - 60])
    bank_table.setStyle(content_table_style)
    elements.append(bank_table)
    elements.append(Spacer(1, 15))
    
    # Customer Information
    customer_info = [
        Paragraph("Customer Information", bold_style),
        Paragraph(f"Client Name: {ctx['account_holder']}", normal_style),
        Paragraph(f"Client number ID: {ctx['client_number']}", normal_style),
        Paragraph(f"Date of birth: {ctx['date_of_birth']}", normal_style),
        Paragraph(f"Account number: {ctx['customer_account_number']}", normal_style),
        Paragraph(f"IBAN: {ctx['customer_iban']}", normal_style),
        Paragraph(f"Bank name: {ctx['customer_bank_name']}", normal_style),
        Paragraph(f"Account Type: {ctx['account_type']}", normal_style)
    ]
    customer_table = Table([[customer_info]], colWidths=[PAGE_WIDTH - 60])
    customer_table.setStyle(content_table_style)
    elements.append(customer_table)
    elements.append(Spacer(1, 15))
    
    # Important Account Information
    info_content = [
        Paragraph("Important Account Information", title_style),
        Table([[None]], colWidths=[PAGE_WIDTH - 60 - 18], rowHeights=[1], style=[
            ('BACKGROUND', (0,0), (-1,-1), colors.HexColor("#D3D3D3")),
            ('LEFTPADDING', (0,0), (-1,-1), 0),
            ('RIGHTPADDING', (0,0), (-1,-1), 0),
        ])
    ]
    if ctx['account_type'] == "Citi Access Checking":
        info_text = (
            "Effective July 1, 2025, the monthly account fee for Citi Access Checking accounts will increase to £10 unless you maintain a minimum daily balance of £1,500 or have qualifying direct deposits of £500 or more per month.\n"
            "Starting June 30, 2025, Citibank will introduce real-time transaction alerts for Citi Access Checking accounts via the Citi Mobile UK app. Enable alerts at citibank.co.uk/alerts.\n"
            "Effective July 15, 2025, Citibank will waive overdraft fees for transactions of £5 or less and cap daily overdraft fees at two per day for Citi Access Checking accounts.\n"
            "For questions, visit citibank.co.uk or contact our Client Contact Centre at 0800 005 555 (or +44 20 7500 5500 from abroad), available 24/7."
        )
    else:
        info_text = (
            "Effective July 1, 2025, the monthly account fee for CitiBusiness Checking accounts will increase to £15 unless you maintain a minimum daily balance of £5,000 or have £2,000 in net purchases on a Citi Business Debit Card per month.\n"
            "Starting June 30, 2025, Citibank will offer enhanced cash flow tools for CitiBusiness Checking accounts via Citi Online Banking, including automated invoice tracking and payment scheduling.\n"
            "Effective July 15, 2025, Citibank will reduce domestic BACS transfer fees to £20 for CitiBusiness Checking accounts, down from £25.\n"
            "For questions, visit citibank.co.uk or contact our Client Contact Centre at 0800 005 555 (or +44 20 7500 5500 from abroad), available 24/7."
        )
    info_content.extend([
        Paragraph(info_text, info_style),
        Table([[None]], colWidths=[PAGE_WIDTH - 60 - 18], rowHeights=[1], style=[
            ('BACKGROUND', (0,0), (-1,-1), colors.HexColor("#D3D3D3")),
            ('LEFTPADDING', (0,0), (-1,-1), 0),
            ('RIGHTPADDING', (0,0), (-1,-1), 0),
        ])
    ])
    info_table = Table([[info_content]], colWidths=[PAGE_WIDTH - 60])
    info_table.setStyle(content_table_style)
    elements.append(info_table)
    elements.append(Spacer(1, 15))
    
    # Transactions Table
    transactions_data = [
        [
            Paragraph("Date", bold_style),  # Changed from "Transaction date" to "Date"
            Paragraph("Information", bold_style),
            Paragraph("Debit", bold_style),
            Paragraph("Credit", bold_style),
            Paragraph("Balance", bold_style)
        ]
    ]
    transactions_data.append([
        Paragraph("", normal_style),
        Paragraph("Opening balance", normal_style),
        Paragraph("", normal_style),
        Paragraph("", normal_style),
        Paragraph(ctx['summary']['opening_balance'], normal_style)
    ])
    for transaction in ctx['transactions']:
        # Truncate description to 25 characters with ellipsis if needed
        desc = transaction["description"]
        if len(desc) > 25:
            desc = desc[:25] + "..."
        transactions_data.append([
            Paragraph(transaction["date"], normal_style),
            Paragraph(desc, normal_style),
            Paragraph(transaction["debit"], normal_style),
            Paragraph(transaction["credit"], normal_style),
            Paragraph(transaction["balance"], normal_style)
        ])
    transactions_data.append([
        Paragraph("", normal_style),
        Paragraph("Total", normal_style),
        Paragraph(ctx['summary']['total_debit'], normal_style),
        Paragraph(ctx['summary']['total_credit'], normal_style),
        Paragraph(ctx['summary']['total'], normal_style)
    ])
    # Calculate total width for centering
    content_width = PAGE_WIDTH - 60
    col_widths = [0.15 * content_width, 0.40 * content_width, 0.12 * content_width, 0.12 * content_width, 0.12 * content_width]
    transactions_table = Table(transactions_data, colWidths=col_widths)
    transactions_table.setStyle(TableStyle([
        ('BACKGROUND', (0,0), (-1,0), colors.HexColor("#D3D3D3")),
        ('BACKGROUND', (0,1), (-1,1), colors.HexColor("#D3D3D3")),
        ('BACKGROUND', (0,-1), (-1,-1), colors.HexColor("#D3D3D3")),
        ('FONT', (0,0), (-1,0), 'Helvetica-Bold', 10),
        ('FONT', (0,1), (-1,-1), 'Helvetica', 9),
        ('TEXTCOLOR', (0,0), (-1,0), colors.HexColor("#003087")),
        ('ALIGN', (2,0), (-1,-1), 'RIGHT'),
        ('GRID', (0,0), (-1,-1), 1, colors.HexColor("#ddd")),
        ('LEFTPADDING', (0,0), (-1,-1), 8),
        ('RIGHTPADDING', (0,0), (-1,-1), 8),
        ('TOPPADDING', (0,0), (-1,-1), 8),
        ('BOTTOMPADDING', (0,0), (-1,-1), 8),
    ]))
    
    # Account Transactions (moved inside the bounding box with KeepTogether)
    # Define meta_table first
    meta_content = [
        Paragraph(ctx['statement_period'], meta_style),
        Paragraph(f"Created on {ctx['statement_date']}", meta_style)
    ]
    meta_table = Table([[meta_content]], colWidths=[PAGE_WIDTH - 60])
    meta_table.setStyle(TableStyle([
        ('ALIGN', (0,0), (-1,-1), 'RIGHT'),
        ('LEFTPADDING', (0,0), (-1,-1), 15),
        ('RIGHTPADDING', (0,0), (-1,-1), 15),
        ('TOPPADDING', (0,0), (-1,-1), 10),
        ('BOTTOMPADDING', (0,0), (-1,-1), 10),
    ]))
    
    transactions_content = [
        [Paragraph("Account Transactions", title_style)],  # Title as a row
        [meta_table],  # Meta table as a row
        [transactions_table]  # Transactions table as a row
    ]
    transactions_content_table = Table(transactions_content, colWidths=[content_width])  # Use the structured 2D list
    transactions_content_table.setStyle(TableStyle([
        ('ALIGN', (0,0), (-1,-1), 'CENTER'),
    ]))
    transactions_content_table.setStyle(content_table_style)
    # Wrap the entire section in KeepTogether to prevent page break
    elements.append(KeepTogether(transactions_content_table))
    
    # Notice
    notice_table = Table([[Paragraph(
        "This printout is for information purposes only. Your regular account statement of assets takes precedence.",
        notice_style
    )]], colWidths=[PAGE_WIDTH - 60])
    notice_table.setStyle(content_table_style)
    elements.append(notice_table)
    
    # Footer
    footer_content = [
        Table([[None]], colWidths=[PAGE_WIDTH - 60], rowHeights=[1], style=[
            ('BACKGROUND', (0,0), (-1,-1), colors.HexColor("#D3D3D3")),
        ]),
        Paragraph("Disclosures", footer_style),
        Paragraph(
            "Citibank UK Limited is authorised by the Prudential Regulation Authority and regulated by the Financial Conduct Authority and the Prudential Regulation Authority.",
            footer_style
        ),
        Paragraph(
            "Our firm’s Financial Services Register number is 805574. Citibank UK Limited is a company limited by shares registered in England and Wales with registered address at Citigroup Centre, Canada Square, Canary Wharf, London E14 5LB.",
            footer_style
        ),
        Paragraph(
            "© 2025 Citibank UK Limited – Calls may be monitored or recorded for training and service quality purposes. CITI, the Arc Design & Citibank are registered service marks of Citigroup Inc.",
            footer_style
        ),
        Paragraph(
            "Deposits are protected by the Financial Services Compensation Scheme (FSCS) up to £85,000 per depositor, subject to eligibility. For details on account terms or overdraft policies, visit citibank.co.uk or call 0800 005 555 (UK) or +44 20 7500 5500 (international).",
            footer_style
        ),
        Paragraph("Citibank", footer_right_style)
    ]
    footer_table = Table([[footer_content]], colWidths=[PAGE_WIDTH - 60], style=[
        ('BACKGROUND', (0,0), (-1,-1), colors.HexColor("#F0F4F8")),
        ('LEFTPADDING', (0,0), (-1,-1), 15),
        ('RIGHTPADDING', (0,0), (-1,-1), 15),
        ('TOPPADDING', (0,0), (-1,-1), 15),
        ('BOTTOMPADDING', (0,0), (-1,-1), 15),
    ])
    elements.append(Spacer(1, 20))
    elements.append(footer_table)
    
    # Build PDF
    doc.build(elements)
    print(f"PDF generated: {output_file}")

In [63]:
# ---------------------------------------------------------------------------------
# Generate the Citibank Classic statement
# ---------------------------------------------------------------------------------

if __name__ == "__main__":
    create_classic_statement(ctx)
    create_template1_statement(ctx)
    create_template2_statement(ctx)

PDF generated: out\citibank_statement_classic.pdf
PDF generated: out\citibank_statement_app_7072.pdf
PDF generated: out\citibank_statement_alternate_7072.pdf
